<img src='Segment.jpg' />

In [513]:
import pandas as pd
import numpy as np
import datetime as dt

In [514]:
data = pd.read_csv('purchases.txt', sep='\t', encoding='utf-8',header=None,  
                       usecols=[0, 1, 2], names=["customers_id", "purchase_amount", "date_of_purchase"], parse_dates=['date_of_purchase'])
data.head()

,customers_id,purchase_amount,date_of_purchase
0,760,25.0,2009-11-06
1,860,50.0,2012-09-28
2,1200,100.0,2005-10-25
3,1420,50.0,2009-07-09
4,1940,70.0,2013-01-25


In [515]:
data['year_of_purchase'] = data['date_of_purchase'].dt.year

# converting to datetime
time1 = pd.Timestamp('2016-01-01')

# And then we're going to compute something a bit specific. 
#We're going to compute the number of days that lapse between 
#January 1st, 2016 and the date of purchase in the data. 
data['days_since'] = (time1 - data['date_of_purchase']).dt.days

data.head()

,customers_id,purchase_amount,date_of_purchase,year_of_purchase,days_since
0,760,25.0,2009-11-06,2009,2247
1,860,50.0,2012-09-28,2012,1190
2,1200,100.0,2005-10-25,2005,3720
3,1420,50.0,2009-07-09,2009,2367
4,1940,70.0,2013-01-25,2013,1071


# Data Preparation

We are going to analysis the Customers based on below 3 factors:
- R (Recency): Number of days since last purchase
- F (Frequency): Number of tracsactions
- M (Monetary): Total amount of transactions (revenue contributed)

In [516]:
# Importing library pandassql
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [517]:
customers_2015 = pysqldf("select customers_id, min(days_since) as 'recêncy', max(days_since) as 'first_purchase', count(*) as 'frequency', avg(purchase_amount) as 'amount' from data group by 1")
customers_2015

,customers_id,recêncy,first_purchase,frequency,amount
0,10,3829,3829,1,30.000000
1,80,343,3751,7,71.428571
2,90,758,3783,10,115.800000
3,120,1401,1401,1,20.000000
4,130,2970,3710,2,50.000000
...,...,...,...,...,...
18412,263820,1,1,1,10.000000
18413,263870,135,135,1,50.000000
18414,263880,34,34,1,20.000000
18415,263890,5,5,1,54.000000


In [518]:
customers_2015.to_csv('customers_2015.csv')

# MANAGERIAL SEGMENTATION 

<img src='customers.jpg' />

In [519]:
# More complex 4-segment solution using which
def recency_lab(customers_2015) :
    
    if customers_2015["recêncy"] > 365*3 :
        return "inactive"
    elif (customers_2015["recêncy"] <= 365*3) & (customers_2015["recêncy"] >= 365*2 ):
        return "cold"
    elif (customers_2015["recêncy"] <= 365*2) & (customers_2015["recêncy"] > 365*1 ):
        return "warm"
    else:
        return 'active'
customers_2015["segment"] = customers_2015.apply(lambda customers_2015:recency_lab(customers_2015),
                                      axis = 1)
customers_2015.head()

,customers_id,recêncy,first_purchase,frequency,amount,segment
0,10,3829,3829,1,30.000000,inactive
1,80,343,3751,7,71.428571,active
2,90,758,3783,10,115.800000,cold
3,120,1401,1401,1,20.000000,inactive
4,130,2970,3710,2,50.000000,inactive


In [520]:
# Quantity of NA and Inactive
customers_2015['segment'].value_counts()

inactive    9158
active      5398
warm        1958
cold        1903
Name: segment, dtype: int64

In [521]:
def f(customers_2015):
    if customers_2015["recêncy"] > 365*3 :
        return "inactive"
    elif (customers_2015["recêncy"] <= 365*3) & (customers_2015["recêncy"] >= 365*2 ):
        return "cold"
    elif (customers_2015['segment'] == 'warm') & (customers_2015["first_purchase"] <= 365*2 ):
        return "new warm"
    elif (customers_2015['segment'] == 'warm') & (customers_2015["amount"] < 100 ):
        return "warm low value"
    elif (customers_2015['segment'] == 'warm') & (customers_2015["amount"] >= 100 ):
        return "warm high value"
    elif (customers_2015['segment'] == 'active') & (customers_2015["first_purchase"] <= 365 ):
        return "new active"
    elif (customers_2015['segment'] == 'active') & (customers_2015["amount"] < 100 ):
        return "active low value"
    else:
        
        return "active high value"
   

customers_2015['segment'] = customers_2015.apply(f, axis=1)
customers_2015.head()

,customers_id,recêncy,first_purchase,frequency,amount,segment
0,10,3829,3829,1,30.000000,inactive
1,80,343,3751,7,71.428571,active low value
2,90,758,3783,10,115.800000,cold
3,120,1401,1401,1,20.000000,inactive
4,130,2970,3710,2,50.000000,inactive


In [522]:
# Show segmentation results
customers_2015['segment'].value_counts()

inactive             9158
active low value     3313
cold                 1903
new active           1512
new warm              938
warm low value        901
active high value     573
warm high value       119
Name: segment, dtype: int64

In [523]:
# Show segmentation results
customers_2015.groupby('segment')['recêncy', 'first_purchase', 'frequency', 'amount'].agg('mean')

,recêncy,first_purchase,frequency,amount
segment,,,,
active high value,88.820244,1985.909250,5.888307,240.045740
active low value,108.361002,2003.801992,5.935406,40.724525
cold,857.781398,1432.117183,2.303205,51.739893
inactive,2178.110832,2546.168377,1.814479,48.112771
new active,84.990741,90.013889,1.045635,77.133847
new warm,509.304904,516.622601,1.044776,66.599026
warm high value,455.126050,2015.352941,4.714286,327.407457
warm low value,474.377358,2063.639290,4.531632,38.591926


# SEGMENTING A DATABASE RETROSPECTIVELY 
<img src='passado.jpg' />

In [524]:
# Compute recency, frequency, and average purchase amount
customers_2014 = pysqldf("select customers_id, min(days_since) - 365 as 'recêncy', max(days_since) - 365 as 'first_purchase', count(*) as 'frequency', avg(purchase_amount) as 'amount' from data where days_since > 365 group by 1")
customers_2014

,customers_id,recêncy,first_purchase,frequency,amount
0,10,3464,3464,1,30.0
1,80,302,3386,6,70.0
2,90,393,3418,10,115.8
3,120,1036,1036,1,20.0
4,130,2605,3345,2,50.0
...,...,...,...,...,...
16900,235200,21,21,1,100.0
16901,235210,89,89,1,500.0
16902,236310,6,6,1,30.0
16903,236660,390,684,2,75.0


In [525]:
# More complex 4-segment solution using which
def recency_lab(customers_2014) :
    
    if customers_2014["recêncy"] > 365*3 :
        return "inactive"
    elif (customers_2014["recêncy"] <= 365*3) & (customers_2014["recêncy"] >= 365*2 ):
        return "cold"
    elif (customers_2014["recêncy"] <= 365*2) & (customers_2014["recêncy"] > 365*1 ):
        return "warm"
    else:
        return 'active'
customers_2014["segment"] = customers_2014.apply(lambda customers_2014:recency_lab(customers_2014),
                                      axis = 1)
customers_2014.head()

,customers_id,recêncy,first_purchase,frequency,amount,segment
0,10,3464,3464,1,30.0,inactive
1,80,302,3386,6,70.0,active
2,90,393,3418,10,115.8,warm
3,120,1036,1036,1,20.0,cold
4,130,2605,3345,2,50.0,inactive


In [526]:
# Quantity of NA and Inactive
customers_2014['segment'].value_counts()

inactive    7512
active      4923
warm        2317
cold        2153
Name: segment, dtype: int64

In [527]:
# Complete segment solution using which, and exploiting previous test as input
def f(customers_2014):
    if customers_2014["recêncy"] > 365*3 :
        return "inactive"
    elif (customers_2014["recêncy"] <= 365*3) & (customers_2014["recêncy"] >= 365*2 ):
        return "cold"
    elif (customers_2014['segment'] == 'warm') & (customers_2014["first_purchase"] <= 365*2 ):
        return "new warm"
    elif (customers_2014['segment'] == 'warm') & (customers_2014["amount"] < 100 ):
        return "warm low value"
    elif (customers_2014['segment'] == 'warm') & (customers_2014["amount"] >= 100 ):
        return "warm high value"
    elif (customers_2014['segment'] == 'active') & (customers_2014["first_purchase"] <= 365 ):
        return "new active"
    elif (customers_2014['segment'] == 'active') & (customers_2014["amount"] < 100 ):
        return "active low value"
    else:
        
        return "active high value"
   

customers_2014['segment'] = customers_2014.apply(f, axis=1)
customers_2014.head()

,customers_id,recêncy,first_purchase,frequency,amount,segment
0,10,3464,3464,1,30.0,inactive
1,80,302,3386,6,70.0,active low value
2,90,393,3418,10,115.8,warm high value
3,120,1036,1036,1,20.0,cold
4,130,2605,3345,2,50.0,inactive


In [528]:
# Show segmentation results
customers_2014['segment'].value_counts()

inactive             7512
active low value     3011
cold                 2153
new active           1437
new warm             1250
warm low value        956
active high value     475
warm high value       111
Name: segment, dtype: int64

In [529]:
# Show segmentation results
customers_2014.groupby('segment')['recêncy', 'first_purchase', 'frequency', 'amount'].agg('mean')# Compute how much revenue is generated by segments

,recêncy,first_purchase,frequency,amount
segment,,,,
active high value,85.338947,1838.050526,5.696842,261.902155
active low value,98.091000,1796.351378,5.633677,40.459174
cold,866.616814,1565.444032,2.254064,51.114605
inactive,2058.441294,2353.036874,1.730964,48.111199
new active,132.086987,142.871260,1.070981,69.725159
new warm,497.317600,504.880000,1.057600,51.367653
warm high value,461.198198,1878.810811,4.414414,187.849110
warm low value,470.661088,1945.426778,4.361925,37.382060


# COMPUTING REVENUE GENERATION PER SEGMENT
<img src='receita.jpg' />

In [530]:
# Compute how much revenue is generated by segments
# Notice that people with no revenue in 2015 do NOT appear
revenues_2015 = pysqldf("select customers_id, sum(purchase_amount) as 'revenue_2015' from data where year_of_purchase = 2015 GROUP BY 1")
revenues_2015

,customers_id,revenue_2015
0,80,80.0
1,480,45.0
2,830,50.0
3,850,60.0
4,860,60.0
...,...,...
5393,263820,10.0
5394,263870,50.0
5395,263880,20.0
5396,263890,54.0


In [531]:
# Merge 2015 customers and 2015 revenue 
actual = pd.merge(customers_2015, revenues_2015, on='customers_id', how="outer")

# Replace values "NaN" by " 0 "
actual['revenue_2015'] = actual['revenue_2015'].replace(np.nan , 0)

actual.head()

,customers_id,recêncy,first_purchase,frequency,amount,segment,revenue_2015
0,10,3829,3829,1,30.000000,inactive,0.0
1,80,343,3751,7,71.428571,active low value,80.0
2,90,758,3783,10,115.800000,cold,0.0
3,120,1401,1401,1,20.000000,inactive,0.0
4,130,2970,3710,2,50.000000,inactive,0.0


In [532]:
# Show average revenue per customer and per segment
actual.groupby('segment')['revenue_2015'].agg('mean')

segment
active high value    323.568935
active low value      52.306043
cold                   0.000000
inactive               0.000000
new active            79.166144
new warm               0.000000
warm high value        0.000000
warm low value         0.000000
Name: revenue_2015, dtype: float64

In [533]:
# Merge 2015 customers and 2015 revenue 
forward = pd.merge(customers_2014, revenues_2015, on='customers_id', how="outer")

# Replace values "NaN" by " 0 "
forward['revenue_2015'] = forward['revenue_2015'].replace(np.nan , 0)

forward.head()

,customers_id,recêncy,first_purchase,frequency,amount,segment,revenue_2015
0,10,3464.0,3464.0,1.0,30.0,inactive,0.0
1,80,302.0,3386.0,6.0,70.0,active low value,80.0
2,90,393.0,3418.0,10.0,115.8,warm high value,0.0
3,120,1036.0,1036.0,1.0,20.0,cold,0.0
4,130,2605.0,3345.0,2.0,50.0,inactive,0.0


In [534]:
# Show average revenue per customer and per segment
forward.groupby('segment')['revenue_2015'].agg('mean')

segment
active high value    254.077895
active low value      41.896556
cold                   6.108221
inactive               2.949466
new active            31.046625
new warm               5.064000
warm high value      114.459459
warm low value        13.494770
Name: revenue_2015, dtype: float64